In [10]:
# Custom function to transform the date in 5 factors dataframe
def to_date(row):
    year = str(row)[:4]
    month = str(row)[4:6]
    day = str(row)[6:]

    return pd.to_datetime('-'.join([year, month, day]))

In [11]:
# Import dependencies
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
# Read the Fama and French factors
factors = pd.read_csv('F-F_Research_Data_5_Factors_2x3_daily.csv')
factors['Date'] = factors.Date.apply(to_date)

In [13]:
# Get the historical data of Microsoft
msft = yf.Ticker("MSFT")
msft = msft.history(period="max")

# Reset the index (Date) for merging
spy.reset_index(inplace=True)
msft.reset_index(inplace=True)

# Calculate the return
spy['Market Return'] = spy['Close'].pct_change(1)
msft['Return'] = msft['Close'].pct_change(1)

In [14]:
# Merging the dataframes and remove row that contains Nan values
df = pd.merge(factors, msft[['Date', 'Return']], on='Date')
df['Return - RF'] = df['Return'] - df['RF']
df.dropna(inplace=True)
df

,Date,Mkt-RF,SMB,HML,RMW,CMA,RF,Return,Return - RF
1,1986-03-14,1.03,-0.83,-0.21,0.11,0.31,0.03,0.035712,0.005712
2,1986-03-17,-0.75,0.00,-0.32,0.38,0.05,0.03,0.017250,-0.012750
3,1986-03-18,0.47,0.04,-0.16,-0.07,0.30,0.03,-0.025431,-0.055431
4,1986-03-19,-0.17,0.16,-0.06,0.15,0.15,0.03,-0.017391,-0.047391
5,1986-03-20,0.39,-0.11,0.02,0.12,0.12,0.03,-0.026547,-0.056547
...,...,...,...,...,...,...,...,...,...
9060,2022-02-22,-1.18,-0.37,0.12,-0.06,0.12,0.00,-0.000729,-0.000729
9061,2022-02-23,-1.96,0.07,1.25,0.00,1.07,0.00,-0.025893,-0.025893
9062,2022-02-24,1.88,0.67,-3.85,-1.74,-1.75,0.00,0.051094,0.051094
9063,2022-02-25,2.23,-0.12,1.20,0.35,-0.19,0.00,0.009233,0.009233


# 5-Factor Model

> ### r<sub>t</sub> - r<sub>t, f</sub> = α + β<sub>mkt</sub>(r<sub>t, mkt</sub> − r<sub>t, f</sub>) + β<sub>SMB</sub>r<sub>t, SMB</sub> + β<sub>HML</sub>r<sub>t,HML</sub> + β<sub>RMW</sub>r<sub>t, RMW</sub> + β<sub>CMA</sub>r<sub>t, CMA</sub> + ε<sub>t</sub>
> <br>

where α<sub>i</sub> is the excess return
      <br>
      SMB (Small Minus Big) is the average return on three small portfolios minus the average return on three big portfolios,

> SMB =	1/3 (Small Value + Small Neutral + Small Growth) - 1/3 (Big Value + Big Neutral + Big Growth).	

HML (High Minus Low) is the average return on two value portfolios minus the average return on two growth portfolios,
 
> HML =	1/2 (Small Value + Big Value) - 1/2 (Small Growth + Big Growth).

RMW (Robust Minus Weak) is the average return on the two robust operating profitability portfolios minus the average return on the two weak operating profitability portfolios,
 	 	 
 	 	
> RMW = 1/2 (Small Robust + Big Robust) - 1/2 (Small Weak + Big Weak).	 
 	 	 
CMA (Conservative Minus Aggressive) is the average return on the two conservative investment portfolios minus the average return on the two aggressive investment portfolios,
 	 	 
 	 	
> CMA = 1/2 (Small Conservative + Big Conservative) - 1/2 (Small Aggressive + Big Aggressive).
      
A rate of return (RoR) is the net gain or loss of an investment over a specified time period, expressed as a percentage of the investment’s initial cost.

Risk free rate is the rate of return of an investment with zero risk.

In [15]:
# Fit data using linear regression
X = df[df.Date.dt.year >= 2017][['Mkt-RF', 'SMB', 'HML','RMW', 'CMA']]
X = X.dropna()
y = df[df.Date.dt.year >= 2017]['Return - RF']

# Add constant term (alpha and error term)
X = sm.add_constant(X)
lr = sm.OLS(y, X)
res = lr.fit()
res.summary()

/Users/hindy/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            Return - RF   R-squared:                       0.773
Model:                            OLS   Adj. R-squared:                  0.772
Method:                 Least Squares   F-statistic:                     878.8
Date:                Sat, 16 Apr 2022   Prob (F-statistic):               0.00
Time:                        17:20:11   Log-Likelihood:                 4348.8
No. Observations:                1298   AIC:                            -8686.
Df Residuals:                    1292   BIC:                            -8655.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0034      0.000    -14.446      0.000      -0.004      -0.003
Mkt-RF         0.0122      0.000     60.351      0.000       0.012       0.013
SMB           -0.0033      0.000     -8.625      0.000      -0.004      -0.003
HML           -0.0039      0.000    -11.821      0.000      -0.005      -0.003
RMW            0.0030      0.001      5.555      0.000       0.002       0.004
CMA           -0.0025      0.001     -3.610      0.000      -0.004      -0.001
==============================================================================
Omnibus:                      201.657   Durbin-Watson:                   1.785
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              994.363
Skew:                           0.630   Prob(JB):                    1.19e-216
Kurtosis:                       7.098   Cond. No.                         3.91
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

R<sup>2</sup> of 77.3% means 77.3% of variance for a dependent variable that's explained by an independent variable or variables.
<br>
Alpha of -0.0034 means the investment underperformed its benchmark index by -0.34%.
<br>
Beta of 0.0122 suggest that the stock has a lower than average risk.